In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import word_tokenize
import string 
import numpy as np
from string import punctuation
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('omw-1.4')
# nltk.download('punkt')
from warnings import filterwarnings
filterwarnings('ignore')
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
raw_articles = pd.read_csv("data/articles.csv")
raw_customers = pd.read_csv("data/customers.csv")
raw_transactions = pd.read_csv("data/transactions_train.csv")

In [3]:
articles = raw_articles.copy()
customers = raw_customers.copy()
transactions = raw_transactions.copy()

In [4]:
articles.head().T

,0,1,2,3,4
article_id,108775015,108775044,108775051,110065001,110065002
product_code,108775,108775,108775,110065,110065
prod_name,Strap top,Strap top,Strap top (1),OP T-shirt (Idro),OP T-shirt (Idro)
product_type_no,253,253,253,306,306
product_type_name,Vest top,Vest top,Vest top,Bra,Bra
product_group_name,Garment Upper body,Garment Upper body,Garment Upper body,Underwear,Underwear
graphical_appearance_no,1010016,1010016,1010017,1010016,1010016
graphical_appearance_name,Solid,Solid,Stripe,Solid,Solid
colour_group_code,9,10,11,9,10
colour_group_name,Black,White,Off White,Black,White


In [5]:
articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105542 entries, 0 to 105541
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   article_id                    105542 non-null  int64 
 1   product_code                  105542 non-null  int64 
 2   prod_name                     105542 non-null  object
 3   product_type_no               105542 non-null  int64 
 4   product_type_name             105542 non-null  object
 5   product_group_name            105542 non-null  object
 6   graphical_appearance_no       105542 non-null  int64 
 7   graphical_appearance_name     105542 non-null  object
 8   colour_group_code             105542 non-null  int64 
 9   colour_group_name             105542 non-null  object
 10  perceived_colour_value_id     105542 non-null  int64 
 11  perceived_colour_value_name   105542 non-null  object
 12  perceived_colour_master_id    105542 non-null  int64 
 13 

In [6]:
articles = articles.dropna()

In [7]:
articles["text"] = articles["prod_name"].map(str) + " " + articles["product_type_name"] +" "+ articles["product_group_name"]+ " "+ articles['graphical_appearance_name']+" "+ articles['colour_group_name'] +" "+ articles['perceived_colour_value_name']+ " " + articles["perceived_colour_master_name"] +" "+ articles["department_name"]+ " "+ articles['index_name']+" "+articles['index_group_name'] +" "+articles['section_name']+ " "+ articles['garment_group_name']+" "+articles['detail_desc']

In [8]:
articles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105126 entries, 0 to 105541
Data columns (total 26 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   article_id                    105126 non-null  int64 
 1   product_code                  105126 non-null  int64 
 2   prod_name                     105126 non-null  object
 3   product_type_no               105126 non-null  int64 
 4   product_type_name             105126 non-null  object
 5   product_group_name            105126 non-null  object
 6   graphical_appearance_no       105126 non-null  int64 
 7   graphical_appearance_name     105126 non-null  object
 8   colour_group_code             105126 non-null  int64 
 9   colour_group_name             105126 non-null  object
 10  perceived_colour_value_id     105126 non-null  int64 
 11  perceived_colour_value_name   105126 non-null  object
 12  perceived_colour_master_id    105126 non-null  int64 
 13 

In [9]:
df_all = articles[['article_id', 'product_code', 'text']]

In [10]:
stop = stopwords.words('english')
stop_words_ = set(stopwords.words('english'))
wn = WordNetLemmatizer()

def black_txt(token):
    return  token not in stop_words_ and token not in list(string.punctuation)  and len(token)>2   
  
def clean_txt(text):
  clean_text = []
  clean_text2 = []
  text = re.sub("'", "",text)
  text=re.sub("(\\d|\\W)+"," ",text) 
  text = text.replace("nbsp", "")
  clean_text = [ wn.lemmatize(word, pos="v") for word in word_tokenize(text.lower()) if black_txt(word)]
  clean_text2 = [word for word in clean_text if black_txt(word)]
  return " ".join(clean_text2)

In [11]:
df_all['text'] = df_all['text'].apply(clean_txt)

In [12]:
df_all['text'].head()

0    strap top vest top garment upper body solid bl...
1    strap top vest top garment upper body solid wh...
2    strap top vest top garment upper body stripe w...
3    shirt idro bra underwear solid black dark blac...
4    shirt idro bra underwear solid white light whi...
Name: text, dtype: object

In [13]:
#!pip install sentence-transformers

Defaulting to user installation because normal site-packages is not writeable
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125940 sha256=5ee4e4d1648d4ef77f0d0d88a51d7d235bbe8b2cc78f2692fd169c5e4058540c
  Stored in directory: c:\users\dvshah\appdata\local\pip\cache\wheels\71\67\06\162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers


In [14]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [15]:
text_list = df_all['text'].tolist()

sentence_embeddings  = sbert_model.encode(text_list[0])
sentence_embeddings.shape

(768,)

In [26]:
text_list[0:5]

['strap top vest top garment upper body solid black dark black jersey basic ladieswear ladieswear womens everyday basics jersey basic jersey top narrow shoulder strap',
 'strap top vest top garment upper body solid white light white jersey basic ladieswear ladieswear womens everyday basics jersey basic jersey top narrow shoulder strap',
 'strap top vest top garment upper body stripe white dusty light white jersey basic ladieswear ladieswear womens everyday basics jersey basic jersey top narrow shoulder strap',
 'shirt idro bra underwear solid black dark black clean lingerie lingeries tights ladieswear womens lingerie nightwear microfibre shirt bra underwired mould lightly pad cup shape bust provide good support narrow adjustable shoulder strap narrow hook eye fasten back without visible seam greater comfort',
 'shirt idro bra underwear solid white light white clean lingerie lingeries tights ladieswear womens lingerie nightwear microfibre shirt bra underwired mould lightly pad cup shape

In [29]:
article_sentences = ["".join(article) for article in text_list]

In [32]:
article_sentences[:7]

['strap top vest top garment upper body solid black dark black jersey basic ladieswear ladieswear womens everyday basics jersey basic jersey top narrow shoulder strap',
 'strap top vest top garment upper body solid white light white jersey basic ladieswear ladieswear womens everyday basics jersey basic jersey top narrow shoulder strap',
 'strap top vest top garment upper body stripe white dusty light white jersey basic ladieswear ladieswear womens everyday basics jersey basic jersey top narrow shoulder strap',
 'shirt idro bra underwear solid black dark black clean lingerie lingeries tights ladieswear womens lingerie nightwear microfibre shirt bra underwired mould lightly pad cup shape bust provide good support narrow adjustable shoulder strap narrow hook eye fasten back without visible seam greater comfort',
 'shirt idro bra underwear solid white light white clean lingerie lingeries tights ladieswear womens lingerie nightwear microfibre shirt bra underwired mould lightly pad cup shape

In [37]:
#result = sbert_model.encode(article_sentences)

In [38]:
#result.shape

In [39]:
#np.save("embedding",result)

In [35]:
result = np.load("data/bert_embedding.npy")

In [36]:
result

array([[ 0.07695773, -0.7046743 , -1.3352145 , ..., -0.28840092,
        -0.18515919, -0.48897102],
       [ 0.11474682, -0.5391601 , -1.4026517 , ...,  0.32394218,
        -0.0243687 , -0.13027613],
       [ 0.16699375, -0.7935319 , -1.4112424 , ...,  0.33043057,
         0.03060157, -0.26536635],
       ...,
       [-0.27468535, -0.8611819 , -1.327831  , ..., -0.8209244 ,
        -0.11368735, -0.26305622],
       [ 0.09037471,  0.18992746, -0.8631339 , ..., -1.0456095 ,
         0.04710359,  0.16714016],
       [ 0.02211573, -0.8925    , -1.4527102 , ...,  0.15218817,
        -0.28289533, -0.27992767]], dtype=float32)

In [69]:
result.reshape(1,-1)

array([[ 0.07695773, -0.7046743 , -1.3352145 , ...,  0.15218817,
        -0.28289533, -0.27992767]], dtype=float32)

In [40]:
transactions = transactions.dropna()

In [41]:
transactions =  transactions.sort_values(by='customer_id')
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id
14479579,2019-07-25,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,745232001,0.021169,1
23574622,2020-03-21,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,890498002,0.031763,2
23574621,2020-03-21,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,887593002,0.021593,2
23574620,2020-03-21,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,841260003,0.011508,2
4212358,2018-12-27,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,625548001,0.044051,1


In [42]:
merged_df = df_all.merge(transactions, how = 'inner', on = ['article_id'])

In [43]:
merged_df.head()

,article_id,product_code,text,t_dat,customer_id,price,sales_channel_id
0,108775015,108775,strap top vest top garment upper body solid bl...,2019-07-14,000f7535bdc611ad136a9f04746d6b1431f50a7f60fbbe...,0.008458,1
1,108775015,108775,strap top vest top garment upper body solid bl...,2019-07-14,000f7535bdc611ad136a9f04746d6b1431f50a7f60fbbe...,0.008458,1
2,108775015,108775,strap top vest top garment upper body solid bl...,2019-07-14,000f7535bdc611ad136a9f04746d6b1431f50a7f60fbbe...,0.008458,1
3,108775015,108775,strap top vest top garment upper body solid bl...,2018-09-29,001ae5408a043f64bccd32beffe2730151414cbdf18a6e...,0.006763,1
4,108775015,108775,strap top vest top garment upper body solid bl...,2018-09-29,001ae5408a043f64bccd32beffe2730151414cbdf18a6e...,0.006763,1


In [44]:
merged_df2 = merged_df.groupby('customer_id', sort=False)['text'].apply(' '.join).reset_index()
merged_df2.head(5)

,customer_id,text
0,000f7535bdc611ad136a9f04746d6b1431f50a7f60fbbe...,strap top vest top garment upper body solid bl...
1,001ae5408a043f64bccd32beffe2730151414cbdf18a6e...,strap top vest top garment upper body solid bl...
2,001ba9e81e13ce12a2585d9ebde923fe74429e9e12ea59...,strap top vest top garment upper body solid bl...
3,0022a721371d5949d174ecba60346d89a9d6c08c0fba4f...,strap top vest top garment upper body solid bl...
4,002323971cbd38fad4512d5114676e5e17eb262db02320...,strap top vest top garment upper body solid bl...


In [45]:
u = "000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318" #customer_id
index = np.where(merged_df2['customer_id'] == u)[0][0]
cust_q = merged_df2.iloc[[index]]
cust_q

,customer_id,text
317613,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,lazer razer brief swimwear bottom swimwear sol...


In [46]:
transactions.loc[transactions['customer_id'] == u]

,t_dat,customer_id,article_id,price,sales_channel_id
24773518,2020-04-18,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,858883002,0.030492,2
24773520,2020-04-18,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,750424014,0.042356,2
24773521,2020-04-18,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,750424014,0.042356,2
24773522,2020-04-18,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,870304002,0.033881,2
24773523,2020-04-18,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,870304002,0.033881,2
24773524,2020-04-18,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,852643001,0.025407,2
24773525,2020-04-18,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,852643003,0.025407,2
21953952,2020-02-03,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,351484002,0.022017,2
21953951,2020-02-03,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,351484002,0.022017,2
24773519,2020-04-18,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,851400006,0.059305,2


In [47]:
def recommendation_product(top, df_all, scores):
    recommendation = pd.DataFrame(columns = ['customer_id', 'article_id',  'product_code', 'detail_desc', 'score'])
    count = 0
    for i in top:
        recommendation.at[count, 'customer_id'] = u
        recommendation.at[count, 'article_id'] = df_all['article_id'][i]
        recommendation.at[count, 'product_code'] = df_all['product_code'][i]
        recommendation.at[count, 'detail_desc'] = articles['detail_desc'][i]   
        recommendation.at[count, 'score'] =  scores[count]
        count += 1
    #print("recommendations generated")        
    return recommendation

In [56]:
cust_list = cust_q['text'].tolist()
user_embeddings  = sbert_model.encode(cust_list)
user_embeddings.reshape(1,-1)

array([[-1.27318844e-01, -7.91413903e-01, -1.35676587e+00,
         4.66501534e-01,  8.91606450e-01, -3.00245494e-01,
         3.74543369e-02, -5.74706554e-01,  3.51013839e-01,
         3.74542087e-01,  5.77386737e-01,  8.73587132e-01,
        -3.08815181e-01,  7.75455683e-02, -1.09578192e-01,
         2.50823438e-01,  5.90778768e-01,  1.66480662e-03,
         2.81401098e-01,  1.94867164e-01, -6.25387371e-01,
        -1.88409358e-01,  5.58481663e-02, -4.82794076e-01,
         1.02236915e+00,  8.58962893e-01,  1.48215204e-01,
         8.61553609e-01,  1.50197148e-01,  5.29901087e-01,
         6.07713163e-01,  5.95571995e-01, -4.13479775e-01,
         5.05680144e-01,  1.15274382e+00,  8.59117746e-01,
        -2.11196095e-02, -2.66492367e-01,  1.91199124e-01,
         2.77442455e-01,  2.03172088e-01, -7.97289193e-01,
         9.26105157e-02, -3.83972734e-01, -1.17017472e+00,
        -5.89617312e-01, -8.21041822e-01,  5.11348188e-01,
        -2.75748670e-02, -3.13990295e-01, -1.12762619e-0

In [66]:
cos_similarity_bert = map(lambda x: cosine_similarity(user_embeddings.reshape(1,-1), x),result.reshape(1,-1))
cos_similarity_bert

In [67]:
output = list(cos_similarity_bert)

ValueError: Expected 2D array, got 1D array instead:
array=[ 0.07695773 -0.7046743  -1.3352145  ...  0.15218817 -0.28289533
 -0.27992767].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.